### Import Relevant Packages

In [1]:
import rasterio
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import geopandas as gpd

In [2]:
# Load Sentinel-2 image
sentinel_image_path = "/home/eoafrica/Sentinel2_AWbasin/outputs_rgb/stacked_rgb.tif"
with rasterio.open(sentinel_image_path) as src:
    sentinel_image = src.read()

In [3]:
# Load ground truth GCP shapefile data
gcp_shapefile_path = "/home/eoafrica/Sentinel2_AWbasin/GCP_LULCawash/lulcgcp.shp"
gcp_data = gpd.read_file(gcp_shapefile_path)

In [4]:
# Extract the values of the image pixels at the locations of the GCPs
gcp_points = gcp_data.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()
gcp_values = []
for point in gcp_points:
    row, col = src.index(point[0], point[1])
    gcp_values.append(sentinel_image[:, row, col])

gcp_values = np.array(gcp_values)

In [6]:
# Extract corresponding class labels from the GCP data
class_labels = gcp_data["class_labe"].values.astype(int)

In [7]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(gcp_values, class_labels, test_size=0.3, random_state=42)

In [8]:
# Define parameter grid for grid search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

In [9]:
# Initialize Support Vector Classifier
svm_classifier = SVC()

In [ ]:
# Perform grid search
grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

In [11]:
# Get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

In [ ]:
# Evaluate the model
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Best Parameters:", best_params)
print("Accuracy:", accuracy)